In [ ]:
!pip install -q g4f

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from copy import deepcopy
from time import sleep
from tqdm import tqdm
import json
import random
import g4f
from time import sleep
import numpy as np
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("huggingface_api_key")
login(hf_token)

In [ ]:
ds = load_dataset('viettelai/wiki-mini-corpus')['train']

In [ ]:
TEMPLATE = """You have been given a passage in Vietnamese:
==PASSAGE==
{passage}
==YOUR TASK==
Your task is to write two type of queries for the passage in JSON format MUST containing the TWO following keys:
- "relevant_{query_type}"
- "irrelevant_{query_type}"
PLEASE adhere to the following guidelines:
- The relevant_{query_type} should be a {query_type} that is related to the passage.
- The irrelevant_{query_type} should be a {query_type} that is NOT related to the passage but can still be in the same topic.
- Try to combine the information in the passage to form the relevant_{query_type}.
- Do not repeat the words in the passage in the relevant_{query_type}, the content needs to be retrieved should be hidden inside the relevant_{query_type}, and SHOULD NOT be too obvious to spot.
- Generated text must be in Vietnamese.
- All the {query_type}s should be at least {query_len} words long.
Be creative in the form of generated text and follow the rules!"""

query_lens = ["7 to 10", "10 to 15", "15 to 20", "20 to 25"]
query_types = ["question", "claim"]


def get_response(passage):
    while True:
        try:
            sleep(3)
            prompt = TEMPLATE.format(
                passage=passage,
                query_len=random.choice(query_lens),
                query_type=random.choice(query_types),
            )
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print("Request failed")
            sleep(15)


In [ ]:
START = 20000
END = START + 500
results = []
N_SAMPLES = 5
SAVE_STEPS = 50
for i, row in enumerate(ds.select(range(START,END)),start=START):
    row = deepcopy(row)
    responses = []
    passage = row['passage']
    for _ in range(N_SAMPLES):
        res = get_response(passage)
        responses.append(res)
    row['responses'] = responses
    results.append(row)
    print(i)
    if (i+1) % SAVE_STEPS == 0:
        with open(f'results_to_{i+1}.json', 'w') as f:
            json.dump(results, f)
        results = []
    